# Creating a model and training it for generating and clustering music

In [23]:
#TODO: Create models: VAE, WAE, U-NET
#Save model weights
#Evaluate in this file?

In [24]:
#libraries
import deeplay as dl
import deeptrack as dt
import torch
import os
from PIL import Image, ImageOps
import torchvision.transforms as T
import matplotlib.pyplot as plt
import numpy as np
from skimage.color import rgb2gray

In [25]:
#Preprocess data
data_dir = os.path.expanduser("./trainImages")

# Load image files using ImageFolder
trainFiles = dt.sources.ImageFolder(root=data_dir)

print(f"Number of train images: {len(trainFiles)}")

Number of train images: 24978


In [26]:
#Create image pipeline

#Create image pipeline
class CropWidth:
    def __init__(self, target_width):
        self.target_width = target_width

    def __call__(self, x: torch.Tensor):
        # assuming input shape [C, H, W]
        return x[..., :self.target_width]
    
image_pip = (dt.LoadImage(trainFiles.path) >> dt.NormalizeMinMax()
             >> dt.MoveAxis(2, 0) >> dt.pytorch.ToTensor(dtype=torch.float) >> CropWidth(2560))

In [27]:
img_tensor = image_pip(trainFiles.path[0])
print(f"The size of each image is: {img_tensor.shape}")

The size of each image is: torch.Size([1, 256, 2560])


In [28]:
#VAE
vae = dl.VariationalAutoEncoder(input_size=(128,2560),
    latent_dim=20, channels=[32, 64, 128],
    reconstruction_loss=torch.nn.BCELoss(), beta=1,
).create()

for i in range(len(vae.encoder.blocks)):
    vae.encoder.blocks[i].layer.kernel_size = (3,30)
    vae.decoder.blocks[i].layer.kernel_size = (3,30)

print(vae)

VariationalAutoEncoder(
  (encoder): ConvolutionalEncoder2d(
    (blocks): LayerList(
      (0): Conv2dBlock(
        (layer): Conv2d(1, 32, kernel_size=(3, 30), stride=(1, 1), padding=(1, 1))
        (activation): ReLU()
      )
      (1): Conv2dBlock(
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (layer): Conv2d(32, 64, kernel_size=(3, 30), stride=(1, 1), padding=(1, 1))
        (activation): ReLU()
      )
      (2): Conv2dBlock(
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (layer): Conv2d(64, 128, kernel_size=(3, 30), stride=(1, 1), padding=(1, 1))
        (activation): ReLU()
      )
      (3): Conv2dBlock(
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (layer): Conv2d(128, 128, kernel_size=(3, 30), stride=(1, 1), padding=(1, 1))
        (activation): Identity()
      )
    )
    (postprocess): Flatten(start_dim=1, end_dim=-

In [29]:
#Giovanni used image_pip & image_pip here, do not really know why // L-Thor
train_dataset = dt.pytorch.Dataset(image_pip & image_pip, inputs=trainFiles)
train_loader = dl.DataLoader(train_dataset, batch_size=4, shuffle=True)

In [30]:
for image, image2 in train_loader:
    print(image.shape)
    break

torch.Size([4, 1, 256, 2560])


In [31]:
#NOTE: Training works, no errors, but it is VERY slow and my computer semi-freezes when I do it, perhaps something is wrong,
#or my computer is trash or maybe we just have to thug it out // L-Thor
vae_trainer = dl.Trainer(max_epochs=2, accelerator="auto")
vae_trainer.fit(vae, train_loader)

c:\Users\Lukas\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightning\pytorch\trainer\configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name                | Type                   | Params | Mode 
-----------------------------------------------------------------------
0 | encoder             | ConvolutionalEncoder2d | 240 K  | train
1 | fc_mu               | Linear                 | 13.1 M | train
2 | fc_var              | Linear                 | 13.1 M | train
3 | fc_dec              | Linear                 | 13.8 M | train
4 | decoder             | ConvolutionalDecoder2d | 326 K  | train
5 | reconstruction_loss | BCELoss                | 0      | train
6 | train_metrics       | MetricCollection       | 0      | train
7 | val_metrics         | MetricCollection       | 0      | train
8 | test_metrics        | MetricCollection       | 0      | train
9 | optimizer           | Adam                   | 0    

Training: |          | 0/? [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (4x1310720 and 655360x20)

In [ ]:
#WAE, let us just focus on the VAE for now, later we can use this code
wae = dl.WassersteinAutoEncoder(
    channels=[32, 64, 128], latent_dim=20,
    reconstruction_loss=torch.nn.MSELoss(reduction="mean"),
).create()

#print(wae)